In [3]:
import cv2
import numpy as np

# Load the image (replace 'image_path' with your actual image path)
image = cv2.imread('../assest/practice/img_edge.png')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Apply thresholding to create a binary image (text vs background)
_, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find contours and hierarchy in the binary image
contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Create an empty mask to draw only the text
mask = np.zeros_like(gray)

# Iterate through contours to draw the outer and inner contours of the text
for i, contour in enumerate(contours):
    # Draw both outer contours and inner contours (holes in letters)
    if hierarchy[0][i][3] == -1:  # Outer contour (text body)
        cv2.drawContours(mask, [contour], -1, 255, cv2.FILLED)

# Perform closing to refine the mask (fill small gaps within the text)
kernel = np.ones((3, 3), np.uint8)
cleaned_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

# Apply the mask to the original image to keep only the text
result = cv2.bitwise_and(image, image, mask=cleaned_mask)

# Convert the result to white text on a black background for clarity
result_inv = cv2.bitwise_not(result, mask=cleaned_mask)

# Display the results
cv2.imshow('Original Image', image)
cv2.imshow('Binary Image', binary)
cv2.imshow('Mask with Text', cleaned_mask)
cv2.imshow('Extracted Text (Final)', result_inv)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [9]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('../assest/practice/img_edge.png')
orig = image.copy()
(H, W) = image.shape[:2]

# Set the new width and height and determine the ratio in change
(newW, newH) = (320, 320)
rW = W / float(newW)
rH = H / float(newH)

# Resize the image and grab the new image dimensions
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]

# Define the layer names that we need from the EAST model
layerNames = [
    "feature_fusion/Conv_7/Sigmoid",
    "feature_fusion/concat_3"]

# Load the pre-trained EAST text detector
net = cv2.dnn.readNet('frozen_east_text_detection.pb')

# Construct a blob from the image and perform a forward pass
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
                             (123.68, 116.78, 103.94), swapRB=True, crop=False)
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)

# Decode the predictions
def decode_predictions(scores, geometry):
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []

    for y in range(0, numRows):
        # Extract the scores and geometrical data
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]

        for x in range(0, numCols):
            # If score is low, ignore
            if scoresData[x] < 0.3:
                continue

            # Compute the offset factor
            offsetX = x * 4.0
            offsetY = y * 4.0

            # Extract the rotation angle
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)

            # Compute the width and height
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]

            # Compute the starting and ending coordinates
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)

            # Append the rectangle and confidence score
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])

    return (rects, confidences)

# Decode the predictions and apply non-maxima suppression
(rects, confidences) = decode_predictions(scores, geometry)
indices = cv2.dnn.NMSBoxes(rects, confidences, 0.5, 0.4)

# Create a mask to hold the text regions
mask = np.zeros(orig.shape[:2], dtype="uint8")

# Loop over the indices
if len(indices) > 0:
    for i in indices.flatten():
        # Scale the bounding box coordinates
        (startX, startY, endX, endY) = rects[i]
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)

        # Draw the bounding box on the mask
        cv2.rectangle(mask, (startX, startY), (endX, endY), 255, -1)

# Apply the mask to the original image
text_only = cv2.bitwise_and(orig, orig, mask=mask)

# Display the result
cv2.imshow('Original Image', orig)
cv2.imshow('Extracted Text', text_only)
cv2.waitKey(0)
cv2.destroyAllWindows()


: 

In [18]:
import cv2
import numpy as np

# Load the image (replace 'image.jpg' with your image file)
image = cv2.imread('../assest/practice/image.png')

# Convert image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply a threshold to get a binary image
_, threshold = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

# Detect contours
contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Loop through each contour
for contour in contours:
    # Approximate the shape of the contour
    epsilon = 0.01 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Determine shape based on the number of vertices
    if len(approx) == 3:
        shape = "Triangle"
    elif len(approx) == 4:
        shape = "Rectangle"
    elif len(approx) > 4:
        shape = "Circle"
    else:
        shape = "Unknown"

    # Calculate area, perimeter, and centroid
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    M = cv2.moments(contour)
    if M['m00'] != 0:
        centroid_x = int(M['m10'] / M['m00'])
        centroid_y = int(M['m01'] / M['m00'])
    else:
        centroid_x, centroid_y = 0, 0

    # Print the shape and its properties
    print(f"Detected shape: {shape}")
    print(f"Area: {area}, Perimeter: {perimeter}, Centroid: ({centroid_x}, {centroid_y})")
    
    # Draw the contours and centroid on the image
    cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
    cv2.circle(image, (centroid_x, centroid_y), 5, (0, 0, 0), -1)
    cv2.putText(image, shape, (centroid_x - 50, centroid_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

# Display the final image with contours and shapes
cv2.imshow("Shape Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Detected shape: Rectangle
Area: 998001.0, Perimeter: 3996.0, Centroid: (499, 499)
Detected shape: Unknown
Area: 668.0, Perimeter: 671.6568541526794, Centroid: (499, 997)
Detected shape: Circle
Area: 42044.0, Perimeter: 765.8376566171646, Centroid: (166, 831)
Detected shape: Triangle
Area: 32851.0, Perimeter: 859.6366448402405, Centroid: (495, 868)
Detected shape: Rectangle
Area: 1321.5, Perimeter: 1325.0710676908493, Centroid: (915, 913)
Detected shape: Rectangle
Area: 1321.5, Perimeter: 1325.0710676908493, Centroid: (83, 913)
Detected shape: Triangle
Area: 32831.0, Perimeter: 859.6366447210312, Centroid: (830, 539)
Detected shape: Rectangle
Area: 53820.5, Perimeter: 925.0710676908493, Centroid: (166, 497)
Detected shape: Circle
Area: 42055.0, Perimeter: 767.4945075511932, Centroid: (496, 495)
Detected shape: Unknown
Area: 658.0, Perimeter: 661.6568541526794, Centroid: (997, 498)
Detected shape: Rectangle
Area: 2.0, Perimeter: 5.656854152679443, Centroid: (333, 334)
Detected shape: Unk